In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
amazon_ratings = pd.read_csv('./drive/MyDrive/archive/ratings_Beauty.csv')

In [7]:
amazon_ratings.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [8]:
amazon_ratings.info()
amazon_ratings.isnull().any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023070 entries, 0 to 2023069
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   UserId     object 
 1   ProductId  object 
 2   Rating     float64
 3   Timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 61.7+ MB


UserId       False
ProductId    False
Rating       False
Timestamp    False
dtype: bool

In [9]:
amazon_ratings.shape

(2023070, 4)

In [10]:
C = amazon_ratings['Rating'].mean()

In [11]:
product_ratings_count = amazon_ratings.groupby('ProductId')['Rating'].count()

In [12]:
product_ratings_mean = amazon_ratings.groupby('ProductId')['Rating'].mean()

In [13]:
product_stats = pd.DataFrame({
    'rating_count': product_ratings_count,
    'rating_mean': product_ratings_mean
})

In [14]:
m = product_stats['rating_count'].quantile(0.90)

In [15]:
qualified_products = product_stats[product_stats['rating_count'] >= m]

In [16]:
def weighted_rating(x, m=m, C=C):
    v = x['rating_count']
    R = x['rating_mean']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [18]:
qualified_products.loc[:, 'weighted_rating'] = qualified_products.apply(weighted_rating, axis=1)

In [19]:
qualified_products = qualified_products.sort_values('weighted_rating', ascending=False)

In [20]:
print(qualified_products.head(10))


            rating_count  rating_mean  weighted_rating
ProductId                                             
B00GJX58PE           398     4.889447         4.862556
B00K7ER6LU            85     4.976471         4.852355
B00I46E8DC           172     4.912791         4.851527
B00F008GFQ           426     4.868545         4.844072
B002YFN49I            98     4.948980         4.842792
B00FPROWWU            79     4.962025         4.832293
B00IBS9QC6           239     4.874477         4.831636
B009OWSHQE            88     4.943182         4.827529
B004CNRDBU            58     5.000000         4.825144
B008DWSJ1O           123     4.894309         4.813301
